In [2]:
# Importing libraries for analysis
import numpy as np
import pandas as pd
import datetime
from dateutil import relativedelta
import calendar
from operator import attrgetter

In [3]:
# Downloading datasets for analysis
customers = pd.read_csv('https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/QXrVDUoQvFhIcg')
orders = pd.read_csv('https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/0zWmh1bYP6REbw')
items = pd.read_csv('https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/xCsQ0FiC-Ue4Lg')

* customers - таблица с уникальными индетификаторами пользователей
* orders - таблица заказов
* items - таблица товарных позиций, входящие в заказ

In [4]:
customers.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [5]:
orders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [6]:
items.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


### Finding the number of users who made only one purchase

In [7]:
# Merging two dataframes
df1 = orders.merge(customers)

In [8]:
# Replacing all NaN values by zero
df1 = df1.fillna(0)

# Filtering of unpaid orders
df1 = df1.query('order_approved_at != 0')

In [9]:
# Grouping purchases by paid orders
df1 = df1.groupby('order_approved_at', as_index=False)\
            .agg({'order_id': 'count'})\
            .rename(columns={'order_id': 'order_count'})

In [10]:
# Finding users who have made only one purchase
single_order = df1.query('order_count == 1').shape[0]
print('The data contains {} users who have made only one purchase'.format(single_order))

The data contains 83688 users who have made only one purchase


### Determining the average number of non-deliverable orders per month

In [11]:
# Formatting the date in database
orders.order_approved_at = pd.to_datetime(orders.order_approved_at)

In [12]:
# Finding the earliest date
start_order_date = orders.order_approved_at.min()

# Find the latest date
end_order_date = orders.order_approved_at.max()

In [13]:
# Determine the number of months contained in the dataset
month_count = round((end_order_date - start_order_date).days / 30)

print('The dataset has {} months'.format(month_count))

The dataset has 24 months


In [14]:
#Let's see the number of orders and information on all order statuses except 'delivered'
orders.query("order_status != 'delivered'")\
    .order_status\
    .value_counts()

shipped        1107
canceled        625
unavailable     609
invoiced        314
processing      301
created           5
approved          2
Name: order_status, dtype: int64

##### *The database contains a large number of undelivered orders to the customers for various reasons. Let's take a closer look at all the reasons for non-delivery of items.*

In [15]:
# Creating a function to determine the average value for different reasons
def check_details(status):
    return round(orders.query("order_status == @status")\
                        .order_status\
                        .count() / month_count, 2)

In [22]:
# Shipment of items from warehouse
status_shipped = check_details('shipped')
print(f"""The average number of orders per month that were not delivered due to the fact 
that the products are shipped from the warehouse and the items are in transit to the recipient = {status_shipped} orders.""")

# Canceled orders
status_canceled = check_details('canceled')
print(f"""\nAverage number of orders per month that were not delivered 
because the recipient canceled the order = {status_canceled} orders.""")

# Unavailable orders
status_unavailable = check_details('unavailable')
print(f"""\nAverage number of orders per month that were not delivered 
because the order is now unavailable = {status_unavailable} orders.""")

# Invoiced orders
status_invoiced = check_details('invoiced')
print(f"""\nAverage number of orders per month that were not delivered 
due to waiting for customer order formation = {status_invoiced} orders.""")

# Processing the assembly of items
status_processing = check_details('processing')
print(f"""\nAverage number of orders per month that were not delivered 
because a customer's order was in processing = {status_processing} orders.""")

# Created orders
status_created = check_details('created')
print(f"""\nAverage number of orders per month that were not delivered
because a customer just created an order = {status_created} orders.""")

# Approved orders
status_approved = check_details('approved')
print(f"""\nAverage number of orders per month that were not delivered
due to customers confirmation = {status_approved} orders.""")

The average number of orders per month that were not delivered due to the fact 
that the products are shipped from the warehouse and the items are in transit to the recipient = 46.12 orders.

Average number of orders per month that were not delivered 
because the recipient canceled the order = 26.04 orders.

Average number of orders per month that were not delivered 
because the order is now unavailable = 25.38 orders.

Average number of orders per month that were not delivered 
due to waiting for customer order formation = 13.08 orders.

Average number of orders per month that were not delivered 
because a customer's order was in processing = 12.54 orders.

Average number of orders per month that were not delivered
because a customer just created an order = 0.21 orders.

Average number of orders per month that were not delivered
due to customers confirmation = 0.08 orders.
